In [1]:
import pandas as pd
import requests
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
from bs4 import BeautifulSoup


def get_salaries(team = False):
    if team == False:
        base = 'https://hoopshype.com/salaries/players/'
    else:
        base = 'https://hoopshype.com/salaries/'
        
    salaries = []
    for i in range(2023,2025):
        if i !=2007:
            url = base+str(i) +'-'+str(i+1)
            if i == 2024:
                url=base

            df = pd.read_html(url)[0]
            #print(df)
            df.drop(columns=['Unnamed: 0'],inplace = True)
            #print(df.columns)
            if i<2022:
                df.columns = ['Player','Salary','Inflation_Salary']
                df['Salary'] = df['Salary'].str.replace('\W', '', regex=True)
                df['Inflation_Salary'] = df['Inflation_Salary'].str.replace('\W', '', regex=True)
            else:
                df = df[df.columns[0:2]]
                df.columns = ['Player','Salary']
                df['Salary'] = df['Salary'].str.replace('\W', '', regex=True)
                df['Inflation_Salary'] = df['Salary']
            df['year'] = i+1
            salaries.append(df)
    final = pd.concat(salaries)
    if team == True:
        final.rename(columns={'Player':'Team'},inplace = True)
    return final
        
def games_per_year():
    url = 'https://www.basketball-reference.com/leagues/NBA_stats_per_game.html'
    games = pd.read_html(url)[0]
    games.columns = games.columns.droplevel()
    games = games.dropna(subset='Season')
    games = games[games['Season']!='Season']
    games['Year'] = games['Season'].str[0:4].astype(int)+1
    games['G'] = games['G'].astype(int)
    games = games[['Year','G']]
    return games
name_map = {'LALakers':'LAL', 'Dallas':'DAL', 'NewYork':'NYK', 'Boston':'BOS', 'Cleveland':'CLE',
         'Orlando':'ORL','SanAntonio':'SAS', 'Denver':'DEN', 'Miami':'MIA', 'Washington':'WAS', 
            'Utah':'UTA',
       'NewOrleans':'NOP', 'Chicago':'CHI', 'Houston':'HOU', 'Charlotte':'CHA', 'Toronto':'TOR',
       'Sacramento': 'SAC', 'Milwaukee':'MIL', 'Indiana':'IND', 'Atlanta':'ATL', 'GoldenState':'GSW',
       'Phoenix':'PHX', 'Philadelphia':'PHI', 'Minnesota':'MIN', 'LAClippers':'LAC', 'Detroit':'DET',
       'Brooklyn':'BKN', 'OklahomaCity':'OKC', 'Memphis':'MEM', 'Portland':'POR'}


#lebron = lebron_master()
salaries = get_salaries(team=False)

#print(salaries)
#payroll = get_salaries(team=True)
lebron = pd.read_csv('lebron.csv')
#lebron['Player'] = lebron['Player'].str.split(' ').str[:-1].str.join(' ')
lebron["Player"]= lebron["Player"].str.lower().str.replace("'", "", regex=True)

lebron['Player'] = lebron['Player'].str.split(' ').str[0].str[0:3] +lebron['Player'].str.split(' ').str[1]

#lebron['Player'] = lebron['Player'].str.split(' ').str[0].str[0]+ lebron['Player'].str.split(' ').str[-1]

#lebron['Player']= lebron['Player'].str.split(' ').str[0].str[0]+ lebron['Player'].str.lower().str.replace('\W', '', regex=True)
lebron['Player']= lebron['Player'].str.lower().str.replace('\W', '', regex=True)

salary = get_salaries(team=False)
print(salary.columns)
extra = pd.DataFrame([['Nick Richards',5000000,5000000,2025]], columns=salary.columns)


salary = pd.concat([salary,extra])

#print(salary)
wrong_names = {'Jose Juan Barea':'J.J. Barea','Timothe Luwawu':'Timothe Luwawu-Cabarrot','Dennis Schroeder':'Dennis Schroder','Didier Ilunga-Mbenga':'DJ Mbenga'
              ,'BJ Boston':'Brandon Boston Jr.'}
salary= salary.replace({'Player':wrong_names})

salary['Name'] = salary['Player']
print(salary[salary.Name.str.contains('Nic')])
#salary['team'] = salary['team'].map(name_map)

#salary['Player'] = salary['Player'].str.split(' ').str[0].str[0]+ salary['Player'].str.split(' ').str[-1]
#salary['Player'] = salary['Player'].str.split(' ').str[0] +salary['Player'].str.split(' ').str[1]


salary["Player"]= salary["Player"].str.lower().str.replace("'", "", regex=True)


salary['Player'] = salary['Player'].str.split(' ').str[0].str[0:3]+salary['Player'].str.split(' ').str[1]

salary['Player']= salary['Player'].str.lower().str.replace('\W', '', regex=True)

#getting player salaries

#payroll = pd.read_csv('team_payroll.csv')
names= dict(zip(lebron.Player,lebron.bref_id))
salary['bref_id'] = salary['Player'].map(names)
salary['bref_id']=np.where(salary.Name =='Markieff Morris','morrima02',salary.bref_id)
#salary['bref_id']=np.where(salary.Name =='Josh Richardson','richani01',salary.bref_id)

#salary['team'] = salary['team'].map(name_map)

#print(names)
lebron.rename(columns={'year':'Year'},inplace = True)
lebron.rename(columns={'team':'Team'},inplace = True)
salary.rename(columns={'year':'Year'},inplace = True)

#salary.rename(columns={'team':'team_name'},inplace = True)
#payroll.rename(columns={'year':'Year'},inplace = True)
#salary = salary.rename(columns={'Player':'Name'})
#getting team payroll
salary = salary[salary.Year>=2010]
salary = salary[salary.Name!='Totals']
salary = salary[salary.Name!='totals']

#salary[salary.bref_id.isnull()]

#salary
salary[salary.bref_id.isnull()]['Name'].unique()
lebron = lebron[lebron.Year==2025]
old_ids = lebron.bref_id.tolist()

Index(['Player', 'Salary', 'Inflation_Salary', 'year'], dtype='object')
                       Player    Salary Inflation_Salary  year  \
135             Nicolas Batum  11710818         11710818  2024   
163           Nicolas Claxton   9625000          9625000  2024   
252  Nickeil Alexander-Walker   4687500          4687500  2024   
330          Andrew Nicholson   2844429          2844429  2024   
362                Nick Smith   2463912          2463912  2024   
56            Nicolas Claxton  27556817         27556817  2025   
247             Nick Richards   5000000          5000000  2025   
256             Nicolas Batum   4668000          4668000  2025   
264  Nickeil Alexander-Walker   4312500          4312500  2025   
353                Nick Smith   2587108          2587108  2025   
0               Nick Richards   5000000          5000000  2025   

                         Name  
135             Nicolas Batum  
163           Nicolas Claxton  
252  Nickeil Alexander-Walker  
330    

In [2]:
salary

,Player,Salary,Inflation_Salary,Year,Name,bref_id
0,stecurry,51915615,51915615,2024,Stephen Curry,curryst01
1,kevdurant,47649433,47649433,2024,Kevin Durant,duranke01
2,nikjokic,47607350,47607350,2024,Nikola Jokic,jokicni01
3,lebjames,47607350,47607350,2024,LeBron James,jamesle01
4,joeembiid,47607350,47607350,2024,Joel Embiid,embiijo01
...,...,...,...,...,...,...
579,anzpasecniks,77500,77500,2025,Anzejs Pasecniks,pasecan01
580,taekinsey,77500,77500,2025,Taevion Kinsey,NaN
581,malhill,75000,75000,2025,Malcolm Hill,NaN
582,eristevenson,66503,66503,2025,Erik Stevenson,NaN


In [3]:
lebron.merge(salary)
new_ids = lebron.bref_id.tolist()
set(old_ids)-set(new_ids)

set()

In [4]:
salary

,Player,Salary,Inflation_Salary,Year,Name,bref_id
0,stecurry,51915615,51915615,2024,Stephen Curry,curryst01
1,kevdurant,47649433,47649433,2024,Kevin Durant,duranke01
2,nikjokic,47607350,47607350,2024,Nikola Jokic,jokicni01
3,lebjames,47607350,47607350,2024,LeBron James,jamesle01
4,joeembiid,47607350,47607350,2024,Joel Embiid,embiijo01
...,...,...,...,...,...,...
579,anzpasecniks,77500,77500,2025,Anzejs Pasecniks,pasecan01
580,taekinsey,77500,77500,2025,Taevion Kinsey,NaN
581,malhill,75000,75000,2025,Malcolm Hill,NaN
582,eristevenson,66503,66503,2025,Erik Stevenson,NaN


In [5]:
lebron = lebron[lebron.Year==2025]

In [6]:

lebron['lastname'] = lebron['Player'].str.split(' ').str[-1]
lebron = lebron.rename(columns={'Team':'team'})
short = lebron[['Year','team','lastname','LEBRON']]
short[short.lastname=='giddens']
null = salary[salary.bref_id.isnull()]
null['lastname'] = null['Name'].str.split(' ').str[-1].str.lower()
null[null.lastname=='giddens']
temp = null.merge(short, on = ['Year','lastname'],how='left')
temp[temp.LEBRON.isnull()]


/tmp/ipykernel_125818/1833644676.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null['lastname'] = null['Name'].str.split(' ').str[-1].str.lower()


,Player,Salary,Inflation_Salary,Year,Name,bref_id,lastname,team,LEBRON
0,xavcooks,1719864,1719864,2024,Xavier Cooks,NaN,cooks,NaN,NaN
1,keyjohnson,559782,559782,2024,Keyontae Johnson,NaN,johnson,NaN,NaN
2,marbolden,502131,502131,2024,Marques Bolden,NaN,bolden,NaN,NaN
3,hendrell,389274,389274,2024,Henri Drell,NaN,drell,NaN,NaN
4,malhill,286583,286583,2024,Malcolm Hill,NaN,hill,NaN,NaN
5,chabediako,215548,215548,2024,Charles Bediako,NaN,bediako,NaN,NaN
6,milnorris,193028,193028,2024,Miles Norris,NaN,norris,NaN,NaN
7,juslewis,173725,173725,2024,Justin Lewis,NaN,lewis,NaN,NaN
8,sirrice,167291,167291,2024,Sir'Jabari Rice,NaN,rice,NaN,NaN
9,andfunk,160857,160857,2024,Andrew Funk,NaN,funk,NaN,NaN


In [7]:
'''
salaries
current = pd.read_csv('../data/pay_table.csv')
lebron= pd.read_csv('lebron.csv')
print(lebron.columns)
print(current.columns)
print(set(current.columns)-set(lebron.columns))
lebron = lebron[lebron.year==2025]
'''

"\nsalaries\ncurrent = pd.read_csv('../data/pay_table.csv')\nlebron= pd.read_csv('lebron.csv')\nprint(lebron.columns)\nprint(current.columns)\nprint(set(current.columns)-set(lebron.columns))\nlebron = lebron[lebron.year==2025]\n"

In [8]:
lebron.columns

Index(['Season', 'Player', 'team', 'Minutes', 'Pos', 'Offensive Archetype',
       'WAR', 'LEBRON', 'O-LEBRON', 'D-LEBRON', 'Year', 'Defensive Role',
       'NBA ID', 'bref_id', 'Games', 'Age', 'lastname'],
      dtype='object')

In [9]:
lebron

,Season,Player,team,Minutes,Pos,Offensive Archetype,WAR,LEBRON,O-LEBRON,D-LEBRON,Year,Defensive Role,NBA ID,bref_id,Games,Age,lastname
7512,2024-25,nikjokić,DEN,1762.075000,C,Post Scorer,10.002889,6.881642,6.418157,0.463485,2025,Anchor Big,203999.0,jokicni01,49.0,29.0,nikjokić
7513,2024-25,shagilgeousalexander,OKC,1804.046667,PG,Shot Creator,9.864108,6.709676,5.754617,0.955059,2025,Helper,1628983.0,gilgesh01,53.0,26.0,shagilgeousalexander
7514,2024-25,jaytatum,BOS,1891.833333,PF,Shot Creator,8.201956,4.504955,3.515674,0.989281,2025,Helper,1628369.0,tatumja01,52.0,26.0,jaytatum
7515,2024-25,antdavis,LAL/DAL,1470.420000,PF,Shot Creator,6.151362,4.248219,2.675766,1.572452,2025,Mobile Big,203076.0,davisan02,43.0,31.0,antdavis
7516,2024-25,giaantetokounmpo,MIL,1431.163333,PF,Shot Creator,5.563875,3.930508,3.457009,0.473499,2025,Helper,203507.0,antetgi01,41.0,30.0,giaantetokounmpo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8040,2024-25,vasmicić,CHO,764.241667,PG,Primary Ball Handler,-0.314285,-3.109136,-1.816439,-1.292697,2025,Chaser,203995.0,micicva01,36.0,31.0,vasmicić
8041,2024-25,corkispert,WAS,1312.526667,SF,Off Screen Shooter,-0.653756,-3.270445,-1.492708,-1.777738,2025,Low Activity,1630557.0,kispeco01,49.0,25.0,corkispert
8042,2024-25,josgreen,CHO,1343.410000,SG,Stationary Shooter,-0.701691,-3.290734,-3.237190,-0.053544,2025,Point of Attack,1630182.0,greenjo02,47.0,24.0,josgreen
8043,2024-25,bilcoulibaly,WAS,1671.970000,SF,Secondary Ball Handler,-1.081684,-3.516966,-2.492859,-1.024108,2025,Point of Attack,1641731.0,coulibi01,50.0,20.0,bilcoulibaly


In [10]:

#salary = salary.drop(columns=['Player'])

salary.rename(columns={'Year':'year'},inplace=True)
salary['Salary']=salary['Salary'].replace('[\$,]', '', regex=True).astype(float)

year_total = salary.groupby('year')['Salary'].sum().reset_index()

games = games_per_year()
print(games.columns)

games.columns = ['year','G']
year_total = year_total.merge(games)
print(year_total['G'])
#year_total['Salary']/=year_total['G']
year_total['Salary']/=1280
year_total.rename(columns ={'Salary':'Win_Cost'},inplace = True)
year_total.drop_duplicates(inplace=True)
pay_table = salary.merge(year_total)
pay_table.fillna(0,inplace = True)
#pay_table['bref_id'].replace({'richani01':'richajo01'}, inplace=True)


#lebron.drop(columns='Player',inplace=True)
pay_table = pay_table.merge(lebron,how='left',on='bref_id')
print(pay_table.columns)
#pay_table['team'] = pay_table['team_x']
#pay_table = pay_table.drop(columns='team')
pay_table['WAR'] = pay_table['WAR'].fillna(0)
pay_table['WAC']= pay_table['WAR'] - (pay_table['Salary']/pay_table['Win_Cost'])
pay_table = pay_table.sort_values(by='WAC')
temp_bron = pd.read_csv('lebron.csv')
id_list = pay_table['NBA ID'].unique().tolist()
temp_bron = temp_bron[~temp_bron['NBA ID'].isin(id_list) ]

#null_names= dict(zip(temp_bron.Player.str.split(' ').str[0],temp_bron.bref_id))
#temp_data = pd.read_csv('players.csv')

#null = null[['Salary','Inflation_Salary','year','team','Name']]
#null['name'] = null.Name.str.split(' ').str[0]
#null['bref_id'] = null['Name'].map(null_names)
#null[null.bref_id.isnull()]


Index(['Year', 'G'], dtype='object')
0    1230
1     814
Name: G, dtype: int64
Index(['Player_x', 'Salary', 'Inflation_Salary', 'year', 'Name', 'bref_id',
       'Win_Cost', 'G', 'Season', 'Player_y', 'team', 'Minutes', 'Pos',
       'Offensive Archetype', 'WAR', 'LEBRON', 'O-LEBRON', 'D-LEBRON', 'Year',
       'Defensive Role', 'NBA ID', 'Games', 'Age', 'lastname'],
      dtype='object')


In [11]:
print(pay_table.columns)

Index(['Player_x', 'Salary', 'Inflation_Salary', 'year', 'Name', 'bref_id',
       'Win_Cost', 'G', 'Season', 'Player_y', 'team', 'Minutes', 'Pos',
       'Offensive Archetype', 'WAR', 'LEBRON', 'O-LEBRON', 'D-LEBRON', 'Year',
       'Defensive Role', 'NBA ID', 'Games', 'Age', 'lastname', 'WAC'],
      dtype='object')


In [12]:
pay_table.columns#pay_table['WAR'] = pay_table['LEBRON WAR']
pay_table['Team']=pay_table['team']
pay_table['Player'] = pay_table['Player_x']
pay_table.drop(columns=[

 'Player_x',
 'Player_y',

 'lastname',
],inplace=True)
pay_table[pay_table.Year==2025]


,Salary,Inflation_Salary,year,Name,bref_id,Win_Cost,G,Season,team,Minutes,...,O-LEBRON,D-LEBRON,Year,Defensive Role,NBA ID,Games,Age,WAC,Team,Player
621,49205800.0,49205800,2025,Kawhi Leonard,leonaka01,4.099381e+06,814,2024-25,LAC,389.866667,...,0.408917,-0.479221,2025.0,Helper,202695.0,15.0,33.0,-11.419603,LAC,kawleonard
7,45640084.0,45640084,2024,Kawhi Leonard,leonaka01,3.848855e+06,1230,2024-25,LAC,389.866667,...,0.408917,-0.479221,2025.0,Helper,202695.0,15.0,33.0,-11.274469,LAC,kawleonard
616,50203930.0,50203930,2025,Bradley Beal,bealbr01,4.099381e+06,814,2024-25,PHO,1197.511667,...,0.015183,-1.083851,2025.0,Point of Attack,203078.0,37.0,31.0,-11.212799,PHO,brabeal
5,46741590.0,46741590,2024,Bradley Beal,bealbr01,3.848855e+06,1230,2024-25,PHO,1197.511667,...,0.015183,-1.083851,2025.0,Point of Attack,203078.0,37.0,31.0,-11.110371,PHO,brabeal
614,51415938.0,51415938,2025,Joel Embiid,embiijo01,4.099381e+06,814,2024-25,PHI,516.045000,...,2.652196,-0.206934,2025.0,Anchor Big,203954.0,17.0,30.0,-10.950626,PHI,joeembiid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,4037277.0,4037277,2024,Payton Pritchard,pritcpa01,3.848855e+06,1230,2024-25,BOS,1506.761667,...,2.265350,-0.090241,2025.0,Point of Attack,1630202.0,54.0,27.0,3.344712,BOS,paypritchard
224,5808435.0,5808435,2024,Tyrese Haliburton,halibty01,3.848855e+06,1230,2024-25,IND,1768.125000,...,2.820422,-0.586472,2025.0,Low Activity,1630169.0,52.0,24.0,3.538373,IND,tyrhaliburton
289,3536280.0,3536280,2024,Alperen Sengun,sengual01,3.848855e+06,1230,2024-25,HOU,1614.876667,...,1.318656,0.899691,2025.0,Mobile Big,1630578.0,52.0,22.0,3.734571,HOU,alpsengun
255,4558680.0,4558680,2024,Jalen Williams,willija06,3.848855e+06,1230,2024-25,OKC,1661.728333,...,1.948769,1.308082,2025.0,Helper,1631114.0,51.0,23.0,4.634010,OKC,jalwilliams


In [13]:
old_pay = pd.read_csv('pay_table.csv')
print((old_pay.columns))


Index(['G', 'Inflation_Salary', 'Salary', 'NBA ID', 'bref_id', 'Year', 'Name',
       'Win_Cost', 'Season', 'Player', 'team', 'Minutes', 'Pos',
       'Offensive Archetype', 'WAR', 'LEBRON', 'O-LEBRON', 'D-LEBRON',
       'Defensive Role', 'Games', 'Age', 'WAC'],
      dtype='object')


In [14]:
pay_table = pay_table.drop_duplicates(subset='bref_id')
filt = pay_table[pay_table.LEBRON.isnull()]
filt

,Salary,Inflation_Salary,year,Name,bref_id,Win_Cost,G,Season,team,Minutes,...,O-LEBRON,D-LEBRON,Year,Defensive Role,NBA ID,Games,Age,WAC,Team,Player
35,33319397.0,33319397,2024,Gordon Hayward,haywago01,3.848855e+06,1230,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.656962,NaN,gorhayward
75,20000000.0,20000000,2024,Bojan Bogdanovic,bogdabo02,3.848855e+06,1230,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.196350,NaN,bojbogdanovic
77,19928571.0,19928571,2024,Joe Harris,harrijo01,3.848855e+06,1230,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.177791,NaN,joeharris
81,18857143.0,18857143,2024,Evan Fournier,fournev01,3.848855e+06,1230,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.899416,NaN,evafournier
96,17000000.0,17000000,2024,Davis Bertans,bertada01,3.848855e+06,1230,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.416897,NaN,davbertans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,92857.0,92857,2024,Demetrius Jackson,jacksde01,3.848855e+06,1230,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.024126,NaN,demjackson
1191,77500.0,77500,2025,Anzejs Pasecniks,pasecan01,4.099381e+06,814,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.018905,NaN,anzpasecniks
607,64343.0,64343,2024,Jalen Crutcher,crutcja01,3.848855e+06,1230,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.016717,NaN,jalcrutcher
610,40459.0,40459,2024,Edmond Sumner,sumneed01,3.848855e+06,1230,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.010512,NaN,edmsumner


In [15]:
pay_table[~pay_table.LEBRON.isnull()]

,Salary,Inflation_Salary,year,Name,bref_id,Win_Cost,G,Season,team,Minutes,...,O-LEBRON,D-LEBRON,Year,Defensive Role,NBA ID,Games,Age,WAC,Team,Player
621,49205800.0,49205800,2025,Kawhi Leonard,leonaka01,4.099381e+06,814,2024-25,LAC,389.866667,...,0.408917,-0.479221,2025.0,Helper,202695.0,15.0,33.0,-11.419603,LAC,kawleonard
616,50203930.0,50203930,2025,Bradley Beal,bealbr01,4.099381e+06,814,2024-25,PHO,1197.511667,...,0.015183,-1.083851,2025.0,Point of Attack,203078.0,37.0,31.0,-11.212799,PHO,brabeal
614,51415938.0,51415938,2025,Joel Embiid,embiijo01,4.099381e+06,814,2024-25,PHI,516.045000,...,2.652196,-0.206934,2025.0,Anchor Big,203954.0,17.0,30.0,-10.950626,PHI,joeembiid
620,49205800.0,49205800,2025,Paul George,georgpa01,4.099381e+06,814,2024-25,PHI,1124.265000,...,-0.101979,0.076133,2025.0,Helper,202331.0,35.0,34.0,-10.288452,PHI,paugeorge
612,55761217.0,55761217,2025,Stephen Curry,curryst01,4.099381e+06,814,2024-25,GSW,1483.106667,...,2.474578,-0.853035,2025.0,Low Activity,201939.0,46.0,36.0,-9.792319,GSW,stecurry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,12160632.0,12160632,2024,Victor Wembanyama,wembavi01,3.848855e+06,1230,2024-25,SAS,1527.253333,...,1.394568,2.366699,2025.0,Mobile Big,1641705.0,46.0,21.0,2.518688,SAS,vicwembanyama
759,11227657.0,11227657,2025,Evan Mobley,mobleev01,4.099381e+06,814,2024-25,CLE,1441.341667,...,1.726875,1.686991,2025.0,Mobile Big,1630596.0,48.0,23.0,2.566368,CLE,evamobley
823,6696429.0,6696429,2025,Payton Pritchard,pritcpa01,4.099381e+06,814,2024-25,BOS,1506.761667,...,2.265350,-0.090241,2025.0,Point of Attack,1630202.0,54.0,27.0,2.760145,BOS,paypritchard
846,5424654.0,5424654,2025,Alperen Sengun,sengual01,4.099381e+06,814,2024-25,HOU,1614.876667,...,1.318656,0.899691,2025.0,Mobile Big,1630578.0,52.0,22.0,3.330072,HOU,alpsengun


In [16]:


filt = pay_table[~pay_table.LEBRON.isnull()]
not_found = set(old_ids)-set(filt.bref_id.tolist())



lebron[lebron.bref_id.isin(not_found)]
index=pd.read_csv('index_master.csv')


index_id=dict(zip(index.bref_id,index.nba_id))

In [17]:

#old_pay = old_pay[old_pay.Year!=2025]
#old_pay.drop(columns='Unnamed: 0',inplace = True)
pay_table.loc[pay_table['Name'] == 'Tidjane Salaun', 'NBA ID'] = 1642275
pay_table.loc[pay_table['Name'] == 'Tidjane Salaun', 'bref_id'] = "salauti01"

pay_table['NBA ID'] = pay_table['bref_id'].map(index_id)

print(pay_table[pay_table['NBA ID'].isna()])

pay_table['NBA ID']=pay_table['NBA ID'].astype(int)


new_pay = pd.concat([old_pay,pay_table])
new_pay['NBA ID'] = new_pay['bref_id'].map(index_id)

print(new_pay[new_pay['NBA ID'].isna()])

new_pay



new_pay


Empty DataFrame
Columns: [Salary, Inflation_Salary, year, Name, bref_id, Win_Cost, G, Season, team, Minutes, Pos, Offensive Archetype, WAR, LEBRON, O-LEBRON, D-LEBRON, Year, Defensive Role, NBA ID, Games, Age, WAC, Team, Player]
Index: []

[0 rows x 24 columns]
         G Inflation_Salary      Salary  NBA ID    bref_id    Year  \
56    1230      $20,250,263  14896000.0     NaN          0  2011.0   
95    1230      $18,397,430  13392000.0     NaN          0  2010.0   
151   1229      $15,257,089  11815850.0     NaN          0  2013.0   
162    990      $14,460,785  11015850.0     NaN          0  2012.0   
202   1230      $13,887,866  10215850.0     NaN          0  2011.0   
...    ...              ...         ...     ...        ...     ...   
5371  1080          $22,987     20000.0     NaN          0  2021.0   
5376  1230          $18,226     18226.0     NaN          0  2023.0   
5387  1230           $7,376      6000.0     NaN          0  2017.0   
8134  1230          2424892   2424892.

,G,Inflation_Salary,Salary,NBA ID,bref_id,Year,Name,Win_Cost,Season,Player,...,WAR,LEBRON,O-LEBRON,D-LEBRON,Defensive Role,Games,Age,WAC,year,Team
0,1230,"$38,644,129",30453000.0,977.0,bryanko01,2014.0,Kobe Bryant,1.654776e+06,2013-14,kobe bryant,...,0.120000,-1.270000,-0.780000,-0.490000,Chaser,6.0,35.0,-18.283096,NaN,NaN
1,1059,"$46,540,848",40231758.0,201939.0,curryst01,2020.0,Stephen Curry,2.785105e+06,2019-20,stephen curry,...,0.360000,1.620000,1.350000,0.280000,Chaser,5.0,31.0,-14.085331,NaN,NaN
2,1230,"$48,325,294",44310840.0,202322.0,walljo01,2022.0,John Wall,3.220158e+06,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1059,"$44,189,315",38199000.0,202322.0,walljo01,2020.0,John Wall,2.785105e+06,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,990,"$27,746,592",21136631.0,1740.0,lewisra02,2012.0,Rashard Lewis,1.575137e+06,2011-12,rashard lewis,...,-0.040000,-2.320000,-2.000000,-0.320000,NaN,28.0,32.0,-13.458917,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,1230,12160632,12160632.0,1641705.0,wembavi01,2025.0,Victor Wembanyama,3.848855e+06,2024-25,vicwembanyama,...,5.678233,3.761267,1.394568,2.366699,Mobile Big,46.0,21.0,2.518688,2024.0,SAS
759,814,11227657,11227657.0,1630596.0,mobleev01,2025.0,Evan Mobley,4.099381e+06,2024-25,evamobley,...,5.305234,3.413866,1.726875,1.686991,Mobile Big,48.0,23.0,2.566368,2025.0,CLE
823,814,6696429,6696429.0,1630202.0,pritcpa01,2025.0,Payton Pritchard,4.099381e+06,2024-25,paypritchard,...,4.393667,2.175109,2.265350,-0.090241,Point of Attack,54.0,27.0,2.760145,2025.0,BOS
846,814,5424654,5424654.0,1630578.0,sengual01,2025.0,Alperen Sengun,4.099381e+06,2024-25,alpsengun,...,4.653358,2.218347,1.318656,0.899691,Mobile Big,52.0,22.0,3.330072,2025.0,HOU


In [18]:
new_pay.to_csv('../web_app/data/pay_table.csv',index=False)

In [19]:
print(pay_table)

         Salary Inflation_Salary  year               Name    bref_id  \
621  49205800.0         49205800  2025      Kawhi Leonard  leonaka01   
616  50203930.0         50203930  2025       Bradley Beal   bealbr01   
614  51415938.0         51415938  2025        Joel Embiid  embiijo01   
620  49205800.0         49205800  2025        Paul George  georgpa01   
612  55761217.0         55761217  2025      Stephen Curry  curryst01   
..          ...              ...   ...                ...        ...   
127  12160632.0         12160632  2024  Victor Wembanyama  wembavi01   
759  11227657.0         11227657  2025        Evan Mobley  mobleev01   
823   6696429.0          6696429  2025   Payton Pritchard  pritcpa01   
846   5424654.0          5424654  2025     Alperen Sengun  sengual01   
255   4558680.0          4558680  2024     Jalen Williams  willija06   

         Win_Cost     G   Season team      Minutes  ...  O-LEBRON  D-LEBRON  \
621  4.099381e+06   814  2024-25  LAC   389.866667  ... 

In [20]:
new_pay['year'].unique()

array([  nan, 2025., 2024.])